<h1><center> Tutorial 5: Pyomo Integration </center></h1>

---

This tutorial shows how to access pyGOLD's standard benchmark problems as Pyomo models. pyGOLD can convert its optimization problems into Pyomo's modeling format, allowing the testing of algorithms that rely on the Pyomo framework. However, at this time pyGOLD does not support automated benchmarking of Pyomo-based optimization algorithms.

### Installation Requirements

To run the example code in this notebook, you'll need Pyomo and the solvers ipopt and glpk. Accessing the problems as Pyomo models only requires the Pyomo package. These dependencies are not required for any other feature of the pyGOLD library.

```bash
conda install -c conda-forge pyomo ipopt glpk
```

## Part 1: The ``.as_pyomo_model()`` Method
---

Problems from pyGOLD's standard problems module can be automatically converted to Pyomo models using the problem's `.as_pyomo_model()` method. This is not a static method. The problem class must be instanciated with desired dimension before this method can be called. For example:

In [1]:
import numpy as np
from pyomo.environ import SolverFactory
import pygold

# Select a problem
problem = pygold.problems.standard.Ackley(3)

# Conver t the problem to a Pyomo model
pyomo_model = problem.as_pyomo_model()

type(pyomo_model)

pyomo.core.base.PyomoModel.ConcreteModel

The ``.as_pyomo_model()`` method handles all parts of constructing the problem as a Pyomo ConcreteModel including setting variable bounds and adding constraint functions, if present. The variable values are initialized to the middle of the bounds.

## Part 2: Example
---


In [3]:
# Choose benchmark problems to use
problems = pygold.get_standard_problems("All")

# Choose pyomo solver to use
solver = SolverFactory('gdpopt')

# Solve each problem with the solver
for problem in problems:
    print(f"\nOptimizing: {problem.__name__}")

    # If problem is n-dimensional, initialize it in 2D
    # Problems must have a specific dimension to be accessed as a pyomo model
    if isinstance(problem.DIM, tuple):
        initialized = problem(2)
    else:
        initialized = problem(problem.DIM)

    # Get the pyomo model
    model = initialized.as_pyomo_model()

    # Solve the problem
    try:
        result = solver.solve(model, algorithm='LOA', tee=False, nlp_solver='ipopt', mip_solver='glpk')
    except Exception as e:
        print(f"Solver failed with error: {e}")
        continue

    # Take the optimal solution
    x_opt = np.array([model.x[i].value for i in range(initialized._ndims)])

    # Output the results
    print(f"Calculated Optimal x: {x_opt}")
    print(f"True Optimal x: {initialized.argmin()}")


Optimizing: Rosenbrock
Calculated Optimal x: [1.00000001 1.00000002]
True Optimal x: [[1.0, 1.0]]

Optimizing: Rothyp
Calculated Optimal x: [3.06640151e-25 3.27680000e-03]
True Optimal x: [[0.0, 0.0]]

Optimizing: HolderTable
Calculated Optimal x: [1.26262726e+00 2.50855166e-23]
True Optimal x: [[8.05502, 9.66459], [-8.05502, -9.66459], [8.05502, -9.66459], [-8.05502, 9.66459]]

Optimizing: Colville
Calculated Optimal x: [1. 1. 1. 1.]
True Optimal x: [[1, 1, 1, 1]]

Optimizing: Matyas
Calculated Optimal x: [1.74706459e-18 1.74706459e-18]
True Optimal x: [[0, 0]]

Optimizing: Schaffer4
Calculated Optimal x: [-1.53483098e-10  2.80241459e+00]
True Optimal x: [[0.0, 1.253115], [0.0, -1.253115], [1.253115, 0.0], [-1.253115, 0.0]]

Optimizing: Perm
Calculated Optimal x: [1.         1.99998185]
True Optimal x: [[1, 2]]

Optimizing: DixonPrice
Calculated Optimal x: [1.         0.70710678]
True Optimal x: [[1.0, 0.7071067811865476]]

Optimizing: Perm0
Calculated Optimal x: [0.47826087 0.978260

## Summary

You've now seen how to convert pyGOLD problems to Pyomo models using `.as_pyomo_model()`.

Next: **Example 1** will demonstrate a complete benchmarking workflow.